Sam Odle <br>
Sept 22, 2021  <br>
Project 1  <br>

In [14]:
import pandas as pd
pd.set_option("display.max_rows", None)

# Part 1: Dataset Download And Extraction
*The dataset consists of feature vectors belonging to 12,330 sessions.
The dataset was formed so that each session
would belong to a different user in a 1-year period to avoid
any tendency to a specific campaign, special day, user
profile, or period.*

##### Load csv correctly into a DataFrame and show contents in a cell

In [15]:
df = pd.read_csv('online_shoppers_intention_cs4364.csv')
print(df.head())

   Administrative  Administrative_Duration  Informational  \
0               0                      0.0              0   
1               0                      0.0              0   
2               0                      0.0              0   
3               0                      0.0              0   
4               0                      0.0              0   

   Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                     0.0             1.0                 0.000000   
1                     0.0             2.0                64.000000   
2                     0.0             1.0                 0.000000   
3                     0.0             2.0                 2.666667   
4                     0.0            10.0               627.500000   

   BounceRates  ExitRates  PageValues  SpecialDay Month  OperatingSystems  \
0         0.20       0.20         0.0         0.0   Feb                 1   
1         0.00       0.10         0.0         0.0   Feb   

##### Holdout dataset split as specified (use rows 10000 - 12331 as our holdout dataset)

In [16]:
df_holdout = df.iloc[10000:12331,]
# print(df_holdout.head())

df_train = df.drop(df.index[10000:12331])
# print(df_train.head())

print(f'Train Shape = {df_train.shape}')
print(f'Holdout Shape = {df_holdout.shape}')
print(f'Initial Shape = {df.shape}')

Train Shape = (10000, 18)
Holdout Shape = (2330, 18)
Initial Shape = (12330, 18)


##### Correct explanation generalization from such a holdout split

Only taking the final 2330 rows as the holdout dataset is problematic for generalization as the dataset appears to be in (roughly) chronological order. This particular holdout set only includes records from November and December.  Retail can be heavily impacted by season trends meaning the holdout set is not representative of the records from other months.

# Part 2: Data Cleaning
##### Use value_counts() in pandas to print out the distributions of the categorical and ordinal numbered features (treat SpecialDay as categorical here). Turn on the setting to reveal missing data -- how many features, and what percent of them, were missing? Discuss in markdown in your notebook.

In [17]:
print(df['Administrative'].value_counts(dropna=False, ascending=True))
print('\n')
print(df['Administrative_Duration'].value_counts(dropna=False, ascending=True, bins = 8))
print('\n')
print(df['Informational'].value_counts(dropna=False, ascending=False))
print('\n')
print(df['Informational_Duration'].value_counts(dropna=False, ascending=True, bins = 8))
print('\n')
print(df['ProductRelated'].value_counts(dropna=False))
print('\n')
print(df['ProductRelated_Duration'].value_counts(dropna=False, ascending=True, bins = 8))
print('\n')
print(df['BounceRates'].value_counts(dropna=False, ascending=True, bins = 8))
print('\n')
print(df['ExitRates'].value_counts(dropna=False, ascending=True, bins = 8))
print('\n')
print(df['PageValues'].value_counts(dropna=False, ascending=True, bins = 6))
print('\n')
print(df['SpecialDay'].value_counts(dropna=False, ascending=True))
print('\n')
print(df['OperatingSystems'].value_counts(dropna=False, ascending=True))
print('\n')
print(df['Browser'].value_counts(dropna=False, ascending=True))
print('\n')
print(df['Region'].value_counts(dropna=False, ascending=True))
print('\n')
print(df['TrafficType'].value_counts(dropna=False, ascending=True))
print('\n')
print(df['Month'].value_counts(dropna=False, ascending=True))
print('\n')
print(df['VisitorType'].value_counts(dropna=False, ascending=True))
print('\n')
print(df['Weekend'].value_counts(dropna=False, ascending=True))
print('\n')
print(df['Revenue'].value_counts(dropna=False, ascending=True))

26       1
27       1
20       2
21       2
23       3
22       4
24       4
19       6
18      12
17      16
16      24
15      38
14      44
13      56
12      86
11     105
10     153
9      225
8      287
7      338
6      432
5      575
4      765
3      915
2     1114
1     1354
0     5768
Name: Administrative, dtype: int64


(2973.906, 3398.75]         1
(2124.219, 2549.062]        3
(2549.062, 2973.906]        3
(1699.375, 2124.219]        8
(1274.531, 1699.375]       36
(849.688, 1274.531]        69
(424.844, 849.688]        321
(-3.4, 424.844]         11889
Name: Administrative_Duration, dtype: int64


0     9699
1     1041
2      728
3      380
4      222
5       99
6       78
7       36
9       15
8       14
10       7
12       5
14       2
16       1
11       1
24       1
13       1
Name: Informational, dtype: int64


(2230.703, 2549.375]        3
(1912.031, 2230.703]        4
(1593.359, 1912.031]       10
(1274.688, 1593.359]       16
(956.016, 1274.688]        34
(637.34

In [22]:
print(df.isna().sum())
missing = df['ProductRelated'].isna().sum()
net = df['ProductRelated'].shape[0]
pct = 100 * missing / net
print(f'Pct Missing from ProductRelated = {round(pct, 2)}%')

Administrative               0
Administrative_Duration      0
Informational                0
Informational_Duration       0
ProductRelated             335
ProductRelated_Duration      0
BounceRates                  0
ExitRates                    0
PageValues                   0
SpecialDay                   0
Month                        0
OperatingSystems             0
Browser                      0
Region                       0
TrafficType                  0
VisitorType                  0
Weekend                      0
Revenue                      0
dtype: int64
Pct Missing from ProductRelated = 2.72%


Only the feature *ProductRelated* has missing data.  2.7% of the values are missing.

##### Use the describe() method in pandas to print out summary statistics. Discuss which features you will have to consider more carefully, based on these results.

In [19]:
des = df.describe()
print(des)
print(' ')

# one indicator of 'a field to keep an eye on' is how many std devs separate the min from the max
for column in des:
    r = des[column]['max'] - des[column]['min']
    num_devs = r / des[column]['std']
    print(f'{des[column].name} - {round(num_devs, 2)}')

       Administrative  Administrative_Duration  Informational  \
count    12330.000000             12330.000000   12330.000000   
mean         2.315166                80.818611       0.503569   
std          3.321784               176.779107       1.270156   
min          0.000000                 0.000000       0.000000   
25%          0.000000                 0.000000       0.000000   
50%          1.000000                 7.500000       0.000000   
75%          4.000000                93.256250       0.000000   
max         27.000000              3398.750000      24.000000   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
count            12330.000000    11995.000000             12330.000000   
mean                34.472398       32.122134              1194.746220   
std                140.749294       44.898778              1913.669288   
min                  0.000000        0.000000                 0.000000   
25%                  0.000000        7.00000

Based on these results, there are a couple fields I will consider more carefully.  At first glance, the *_Duration* fields appear to have much higher means and much greater variation.  These may be candidates for scaling/normalization.
<br>
To get a better sense of the data I took a look at the number of standard deviations that comprise the range for each series.  This quick analysis showed that *PageValues*, *Informational*, and *ProductRelated* also have a bit of variation that may need to be dealt with.

##### Handle any missing data in your training data, but do not simply delete the rows. In your notebook, discuss why you chose to handle the missing data that way.


##### The holdout dataset also contains missing data. Discuss and implement how you recovered those items, without deleting those rows.
*In this instance, the holdout data does not include any missing data.  If it did, I would've handled it the same way I handled the missing data in the training set.*


##### Discuss (and implement if applicable) whether or not you need to scale/normalize your features, and which ones, if any.

##### There are several categorical features. Discuss and implement if you will encode them as ordinal numbers, or one-hot encode them, and why you chose to do so for each such feature.

##### You don't need to implement this, but in the dataset, were there any ordinal features that they authors should have recorded as categorical, in your opinion? Why or why not? Discuss in a markdown cell.

# Part 3: Feature Simplification And Engineering
##### Use a heatmap to show the correlation between all feature pairs. Discuss, if any, which features you would recommend dropping from your model. Also discuss why you would want to drop them (what is the expected benefit?)	5 points

##### Given what you know about the limitations of RandomForests, engineer one additional feature, and discuss why you think it might help the model.

# Part 4: Model Training

##### Separate your training data into features and labels (X and y).

##### The labels for this dataset are highly imbalanced. Discuss and implement how you will handle this situation for this analysis.

##### Instantiate a RandomForest model of your choosing.

##### Define a grid to tune at least three different hyperparameters with at least two different values each. Discuss why you think these parameter values might be useful for this dataset.

##### Set up a gridsearchCV with 5-fold cross validation. Discuss what accuracy metric you chose and why.

##### Train your model using gridsearchCV, and report the best performing hyperparameters.

# Part 5: Model Analysis

##### Calculate accuracy, precision and recall on the holdout dataset. Discuss which metric you think is most meaningful for this dataset, and why

##### Discuss how the model performance on holdout compares to the model performance during training. Do you think your model will generalize well? Why or why not?

##### Generate a confusion matrix and discuss your results.

##### Print out the feature importances of your model.

# Part 6: Model Comparison

##### Train and tune another decision-tree based model on your training dataset. Using the best performing hyperparameters, test this model on your holdout. How did it perform, compared to your earlier model? Do you think your results will generalize?

##### Next, repeat training and tuning on the same data with a LogisticRegression model. Do you need to do any additional feature cleaning or scaling here? Why or why not?

# Extra Credit